<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-A/blob/main/BFH-25-Tabbed-Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IFC Tabbed Dashboard

This notebook creates an interactive dashboard with [Dash](https://dash.plotly.com/) inside Jupyter. It demonstrates how to analyze IFC data and visualize the results in an interative way.

**What you'll learn:**
- Load IFC files from GitHub or upload your own
- Extract building elements, storeys, and materials
- Visualize data in an interactive tabbed dashboard
- Identify unassigned elements and data quality issues


## 1. Install Dependencies

Run this cell to install required Python packages. This is needed in Colab or fresh environments.


In [11]:
%pip install ifcopenshell pandas plotly dash ipywidgets


## 2. Load IFC Model

Choose how to load your IFC model:
- **Option A**: Download from GitHub repository (works in Colab)
- **Option B**: Upload your own IFC file

The code extracts:
- Building storeys (from `IfcRelContainedInSpatialStructure`)
- Element types (e.g., `IfcWall`, `IfcDoor`, `IfcWindow`)
- Materials (from `IfcRelAssociatesMaterial`)

Result: A clean DataFrame with one row per element-material combination.


In [12]:
import ifcopenshell
import ifcopenshell.util.element as uel
import pandas as pd

# IFC-Datei öffnen
model = ifcopenshell.open(local_filename)

element_data = []

for element in model.by_type('IfcElement'):
    element_id = element.GlobalId
    element_type = element.is_a()
    element_name = getattr(element, 'Name', None) or 'Unnamed'

    # Property Sets extrahieren
    psets = uel.get_psets(element)

    # Fire Rating Property suchen
    fire_rating = None
    for pset_name, properties in psets.items():
        possible_names = ['FireRating', 'fire_rating', 'FireRatingName', 'FireResistance']
        for prop_name, prop_value in properties.items():
            if prop_name in possible_names:
                fire_rating = prop_value
                break
        if fire_rating:
            break

    # Geschoss finden
    storey = 'Not assigned'
    for rel in model.by_type('IfcRelContainedInSpatialStructure'):
        if element in rel.RelatedElements:
            if rel.RelatingStructure and rel.RelatingStructure.is_a('IfcBuildingStorey'):
                storey = rel.RelatingStructure.LongName or rel.RelatingStructure.Name
                break

    element_data.append({
        'ElementId': element_id,
        'ElementType': element_type,
        'ElementName': element_name,
        'FireRating': fire_rating or 'Nicht klassifiziert',
        'Storey': storey
    })

df = pd.DataFrame(element_data)
df.head()


,ElementId,ElementType,ElementName,FireRating,Storey
0,3ZT5EhLQn6JfRlijT8hpjb,IfcBeam,Concrete-Rectangular Beam:CB24x24:606873,R 90,L1_35_Low
1,3ZT5EhLQn6JfRlijT8hpjd,IfcBeam,Concrete-Rectangular Beam:CB24x24:606875,R 90,L1_35_Low
2,3ZT5EhLQn6JfRlijT8hpjX,IfcBeam,Concrete-Rectangular Beam:CB24x24:606877,R 90,L1_37_Med
3,3ZT5EhLQn6JfRlijT8hpjZ,IfcBeam,Concrete-Rectangular Beam:CB24x24:606879,R 90,L1_37_Med
4,3ZT5EhLQn6JfRlijT8hpjT,IfcBeam,Concrete-Rectangular Beam:CB24x24:606881,R 90,L1_43_High


## 3. Prepare Data for Visualization

Create aggregated DataFrames for the dashboard:
- **storey_summary**: Element counts per storey
- **storey_element_breakdown**: Element types per storey
- **storey_material_breakdown**: Materials per storey
- **material_element_breakdown**: Material-element combinations
- **unassigned_by_type**: Elements without storey assignment


In [13]:
storey_summary = (
    ifc_df.groupby('Storey')['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values('ElementCount', ascending=False)
)

storey_element_breakdown = (
    ifc_df.groupby(['Storey', 'ElementType'])['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values(['Storey', 'ElementCount'], ascending=[True, False])
)

storey_material_breakdown = (
    ifc_df.groupby(['Storey', 'Material'])['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values(['Storey', 'ElementCount'], ascending=[True, False])
)

material_element_breakdown = (
    ifc_df.groupby(['Material', 'ElementType'])['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values('ElementCount', ascending=False)
)

# Unassigned elements analysis
unassigned_elements = ifc_df[ifc_df['Storey'] == 'Not assigned'].copy()

unassigned_by_type = (
    unassigned_elements.groupby('ElementType')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

storey_summary.head()


,Storey,ElementCount
9,Not assigned,717
0,Elevator Pit,166
11,Parking,121
5,L3,104
4,L2,100


## 4. Build Interactive Dashboard

Creates a Dash app with multiple tabs:
- **Storey Overview**: Element counts and type distribution
- **Material Insights**: Material usage and combinations
- **Unassigned Elements**: Quality control for spatial organization

The dashboard runs directly in the notebook using Plotly for interactive charts.


In [14]:
from dash import Dash, dcc, html
import plotly.express as px

# --- Daten für Diagramme vorbereiten ---
fire_rating_counts = (
    df.groupby('FireRating')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

fire_rating_by_type = (
    df.groupby(['FireRating', 'ElementType'])['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

fire_rating_by_storey = (
    df.groupby(['FireRating', 'Storey'])['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

# --- Diagramme ---
fig_fire_overview = px.bar(
    fire_rating_counts,
    x='FireRating',
    y='Count',
    title='Fire Rating Verteilung',
    text_auto='.0f'
)

fig_fire_by_type = px.bar(
    fire_rating_by_type,
    x='ElementType',
    y='Count',
    color='FireRating',
    title='Fire Rating nach Elementtyp'
)

fig_fire_by_storey = px.bar(
    fire_rating_by_storey,
    x='Storey',
    y='Count',
    color='FireRating',
    title='Fire Rating nach Geschoss'
)

fig_heatmap = px.density_heatmap(
    df[df['FireRating'] != 'Nicht klassifiziert'],
    x='ElementType',
    y='FireRating',
    title='Fire Rating vs Elementtyp'
)

# --- Dashboard Layout ---
app = Dash(__name__)

app.layout = html.Div([
    html.H1('🔥 Fire Rating Dashboard – Gruppe A'),
    html.P('Analyse der Brandschutzklassifikation aus dem IFC-Modell'),

    dcc.Tabs([
        dcc.Tab(label='Übersicht', children=[
            html.H3('Gesamtverteilung'),
            dcc.Graph(figure=fig_fire_overview)
        ]),
        dcc.Tab(label='Nach Elementtyp', children=[
            html.H3('Fire Rating nach Bauteiltyp'),
            dcc.Graph(figure=fig_fire_by_type)
        ]),
        dcc.Tab(label='Nach Geschoss', children=[
            html.H3('Fire Rating pro Geschoss'),
            dcc.Graph(figure=fig_fire_by_storey)
        ]),
        dcc.Tab(label='Heatmap', children=[
            html.H3('Fire Rating Heatmap'),
            dcc.Graph(figure=fig_heatmap)
        ])
    ])
])

app.run(jupyter_mode='inline', height=900, port=8050)


<IPython.core.display.Javascript object>

## 5. Next Steps

**Extend the dashboard:**
- Add dropdown filters for specific storeys or materials
- Export data to CSV for reporting
- Update some data (eg. calculate cost or emissions)

**Try different models:**
- Upload your own IFC files using Option B
- Compare different buildings or design versions
- Analyze data quality and completeness
